Same setup as before


In [1]:
import os

os.chdir("../..")
import django_init


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# For better printing

from rich import print
from rich.console import Console
from rich.table import Table

console = Console()

from django.db import connection

In [4]:
from utils.perf_display import format_duration, perf_counter

In [5]:
from books.models import Library, Review, Book
from django.db.models import Count

public = Library.objects.order_by("-id").first()
public_id = public.id

# Get the library with the most books
alexandria_id, book_count = (
    Book.objects.values("library_id")
    .annotate(book_count=Count("id"))
    .order_by("-book_count")
    .values_list("library_id", "book_count")
    .first()
)

alexandria = Library.objects.get(id=alexandria_id)

---

## 2nd exercice

Let's continue with another usual example.
We want to implement a standard endpoint returning a page of filtered and ordered reviews.

A minimal implementation is available in [this file]("../../../../books/views/review/simple.py).


In [6]:
# Let's try the existing endpoint
from rest_framework.test import APIClient

client = APIClient()
result = client.get(f"/reviews/{alexandria_id}/simple")


/home/laurent/.local/share/virtualenvs/django-perfs-playground-tFy-cSse/lib/python3.10/site-packages/rest_framework/pagination.py:200: UnorderedObjectListWarning: Pagination may yield inconsistent results with an unordered object_list: <class 'books.models.review.Review'> QuerySet.
  paginator = self.django_paginator_class(queryset, page_size)


<br>
<br>
<br>


---

Let's try two different orderings:

- by descending `written_at` datetime
- by ascending `id`


In [7]:
with perf_counter(message=f"ordering: -written_at", time_sql=True, print_sql=True):
    result = client.get(f"/reviews/{alexandria_id}/ordered?ordering=-written_at")

ordering: -written_at
  Total: 16.65
  SQL: 16.64 

      (duration: 16.644)   SELECT "books_review"."id", "books_review"."book_id", "books_review"."reader_id", 
"books_review"."rating", "books_review"."comments", "books_review"."written_at", "books_review"."library_id" FROM 
"books_review" WHERE "books_review"."library_id" = 6 ORDER BY "books_review"."written_at" DESC LIMIT 20

In [8]:
with perf_counter(message=f"ordering: id", time_sql=True, print_sql=True):
    result = client.get(f"/reviews/{alexandria_id}/ordered?ordering=id")

ordering: id
  Total: 0.02
  SQL: 0.01 

      (duration: 0.009)   SELECT "books_review"."id", "books_review"."book_id", "books_review"."reader_id", 
"books_review"."rating", "books_review"."comments", "books_review"."written_at", "books_review"."library_id" FROM 
"books_review" WHERE "books_review"."library_id" = 6 ORDER BY "books_review"."id" ASC LIMIT 20

The SQL queries are exactly the same, except for the `ORDER BY` clause, and yet, they have a really different execution time.

We won't be able to improve the overall duration by improving the query (while keeping the expected behaviour).
<br>

Let's get a better summary of ordering duration.


In [9]:
from books.tests.benchmarks import benchmark_list_reviews

# benchmark the ordering endpoint
benchmark_list_reviews(
    ["simple-list-reviews", "ordered-list-reviews"],
    library_ids=[public_id, alexandria_id],
)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Query params                ┃ Library    ┃ duration  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│                             │ Public     │  0.008 s  │
├─────────────────────────────┼────────────┼───────────┤
│                             │ Alexandria │  0.006 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=id                 │ Public     │  23.688 s │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=id                 │ Alexandria │  0.014 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=written_at         │ Public     │  0.949 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=written_at         │ Alexandria │  10.748 s │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=rating             │ Public     │  0.127 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=rating             │ Alexandria │  9.138 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=-id                │ Public     │  0.337 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=-id                │ Alexandria │  0.005 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=-written_at        │ Public     │  0.689 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=-written_at        │ Alexandria │  2.864 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=-rating            │ Public     │  0.108 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=-rating            │ Alexandria │  2.655 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=-written_at,rating │ Public     │  0.124 s  │
├─────────────────────────────┼────────────┼───────────┤
│ ordering=-written_at,rating │ Alexandria │  2.236 s  │
└─────────────────────────────┴────────────┴───────────┘

A few remarks about these results:

- All these orderings are quite fast for `Public`, and all about the same duration
- `Alexandria` is usually slower than `Public`, except for `id`, which is significantly faster
- All `Alexandria` orderings are slow, except `id`

We saw previously we cannot explain these differences just by looking at the SQL query.

<br>
<br>

We'll now use a new tool: the **`EXPLAIN ANALYZE` SQL statement**. This will display the exact internal operations used by the PostgreSQL engine.

In particular, we'll be able to inspect which indexes are used by each query.


In [10]:
def explain_qs(library_id, ordering):
    with perf_counter(message=f"ordering: {ordering}", time_sql=True, print_sql=True):
        result = (
            Review.objects.filter(library_id=library_id)
            .order_by(ordering)[:20]
            .explain(analyze=True)
        )
    print(result)
    console.rule()

In [11]:
explain_qs(alexandria_id, "-written_at")

ordering: -written_at
  Total: 2.55
  SQL: 2.55 

      (duration: 2.548)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY 
"books_review"."written_at" DESC LIMIT 20

Limit  (cost=1447614.65..1447616.98 rows=20 width=211) (actual time=2534.229..2542.902 rows=20 loops=1)
  ->  Gather Merge  (cost=1447614.65..1631039.59 rows=1572104 width=211) (actual time=2465.210..2473.879 rows=20 
loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Sort  (cost=1446614.62..1448579.75 rows=786052 width=211) (actual time=2438.488..2438.491 rows=16 
loops=3)
              Sort Key: written_at DESC
              Sort Method: top-N heapsort  Memory: 35kB
              Worker 0:  Sort Method: top-N heapsort  Memory: 38kB
              Worker 1:  Sort Method: top-N heapsort  Memory: 38kB
              ->  Parallel Seq Scan on books_review  (cost=0.00..1425698.06 rows=786052 width=211) (actual 
time=91.790..2202.327 rows=675591 loops=3)
                    Filter: (library_id = 6)
                    Rows Removed by Filter: 12672873
Planning Time: 0.176 ms
JIT:
  Functions: 13
  Options: Inlining true, Optimization true, Expressions true, Deforming true
  Timing: Generation 3.808 ms, Inlining 191.713 ms, Optimization 93.883 ms, Emission 58.421 ms, Total 347.825 ms
Execution Time: 2544.114 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Lot of information here. What can we notice?

- Planning time is less than a millisecond

- Execution time is around 10 seconds

- The main operation is a `Parallel Seq Scan on books_review`, i.e. the whole `books_review` table is read, and only the rows matching `(library_id = 6)` are kept. `12666173` rows are filtered out

- Finally, a heapsort is used to get the first 20 rows


Conclusion?
<br>
<br>
<br>


<br>
<br>
<br>


Our database is probably missing some indexes:

- `written_at` column
- `rating` column


In [12]:
from utils.sql import use_indexes

with use_indexes("review_written_at_idx"):
    explain_qs(alexandria_id, "-written_at")
    explain_qs(alexandria_id, "written_at")


ordering: -written_at
  Total: 0.01
  SQL: 0.01 

      (duration: 0.006)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY 
"books_review"."written_at" DESC LIMIT 20

Limit  (cost=0.44..1043.93 rows=20 width=211) (actual time=1.298..3.346 rows=20 loops=1)
  ->  Index Scan Backward using review_written_at_idx on books_review  (cost=0.44..98428448.77 rows=1886525 
width=211) (actual time=1.295..3.338 rows=20 loops=1)
        Filter: (library_id = 6)
        Rows Removed by Filter: 305
Planning Time: 2.117 ms
Execution Time: 3.392 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ordering: written_at
  Total: 0.01
  SQL: 0.00 

      (duration: 0.003)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY 
"books_review"."written_at" ASC LIMIT 20

Limit  (cost=0.44..1043.93 rows=20 width=211) (actual time=0.688..2.135 rows=20 loops=1)
  ->  Index Scan using review_written_at_idx on books_review  (cost=0.44..98428448.77 rows=1886525 width=211) 
(actual time=0.686..2.126 rows=20 loops=1)
        Filter: (library_id = 6)
        Rows Removed by Filter: 323
Planning Time: 0.148 ms
Execution Time: 2.201 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

So far, so good, ordering by `written_at` is now fast enough.

<br>
<br>
<br>


In [13]:
with use_indexes("review_rating_idx"):
    explain_qs(alexandria_id, "rating")
    explain_qs(alexandria_id, "-rating")

ordering: rating
  Total: 0.00
  SQL: 0.00 

      (duration: 0.003)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY 
"books_review"."rating" ASC LIMIT 20

Limit  (cost=0.44..991.32 rows=20 width=211) (actual time=0.895..1.104 rows=20 loops=1)
  ->  Index Scan using review_rating_idx on books_review  (cost=0.44..93465762.91 rows=1886525 width=211) (actual 
time=0.893..1.100 rows=20 loops=1)
        Filter: (library_id = 6)
        Rows Removed by Filter: 481
Planning Time: 0.872 ms
Execution Time: 1.137 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ordering: -rating
  Total: 4.18
  SQL: 4.17 

      (duration: 4.174)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY 
"books_review"."rating" DESC LIMIT 20

Limit  (cost=0.44..991.32 rows=20 width=211) (actual time=4171.901..4172.463 rows=20 loops=1)
  ->  Index Scan Backward using review_rating_idx on books_review  (cost=0.44..93465762.91 rows=1886525 width=211) 
(actual time=4171.899..4172.454 rows=20 loops=1)
        Filter: (library_id = 6)
        Rows Removed by Filter: 1819598
Planning Time: 0.106 ms
Execution Time: 4172.543 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

`rating` is ok, but `-rating` is still too slow

```sql
CREATE INDEX review_library_id_rating_idx ON public.books_review USING btree (library_id, rating)
```


<br>
<br>
<br>


<br>
<br>
<br>


In [14]:
with use_indexes("review_library_id_rating_idx"):
    explain_qs(alexandria_id, "rating")
    explain_qs(alexandria_id, "-rating")

ordering: rating
  Total: 0.01
  SQL: 0.01 

      (duration: 0.005)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY 
"books_review"."rating" ASC LIMIT 20

Limit  (cost=0.56..36.07 rows=20 width=211) (actual time=1.446..1.496 rows=20 loops=1)
  ->  Index Scan using review_library_id_rating_idx on books_review  (cost=0.56..3348862.04 rows=1886525 width=211)
(actual time=1.443..1.490 rows=20 loops=1)
        Index Cond: (library_id = 6)
Planning Time: 1.966 ms
Execution Time: 1.540 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ordering: -rating
  Total: 0.00
  SQL: 0.00 

      (duration: 0.002)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY 
"books_review"."rating" DESC LIMIT 20

Limit  (cost=0.56..36.07 rows=20 width=211) (actual time=0.769..0.827 rows=20 loops=1)
  ->  Index Scan Backward using review_library_id_rating_idx on books_review  (cost=0.56..3348862.04 rows=1886525 
width=211) (actual time=0.768..0.823 rows=20 loops=1)
        Index Cond: (library_id = 6)
Planning Time: 0.102 ms
Execution Time: 0.871 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Ok, seems goood now!
<br>
<br>
<br>


<br>
<br>
<br>


<br>
<br>
<br>


---

Let's now investigate the weird results on the `id` / `-id` ordering


In [15]:
explain_qs(public_id, "id")
explain_qs(public_id, "-id")
explain_qs(alexandria_id, "id")
explain_qs(alexandria_id, "-id")


ordering: id
  Total: 6.42
  SQL: 6.42 

      (duration: 6.417)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 10015 ORDER BY 
"books_review"."id" ASC LIMIT 20

Limit  (cost=0.44..280.67 rows=20 width=211) (actual time=6413.715..6415.238 rows=20 loops=1)
  ->  Index Scan using books_review_pkey on books_review  (cost=0.44..2860175.68 rows=204129 width=211) (actual 
time=6413.714..6415.234 rows=20 loops=1)
        Filter: (library_id = 10015)
        Rows Removed by Filter: 20004693
Planning Time: 0.220 ms
Execution Time: 6415.268 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ordering: -id
  Total: 0.11
  SQL: 0.11 

      (duration: 0.110)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 10015 ORDER BY 
"books_review"."id" DESC LIMIT 20

Limit  (cost=0.44..280.67 rows=20 width=211) (actual time=108.751..109.198 rows=20 loops=1)
  ->  Index Scan Backward using books_review_pkey on books_review  (cost=0.44..2860175.68 rows=204129 width=211) 
(actual time=108.750..109.193 rows=20 loops=1)
        Filter: (library_id = 10015)
        Rows Removed by Filter: 41442
Planning Time: 0.097 ms
Execution Time: 109.263 ms

...

Limit  (cost=0.44..30.76 rows=20 width=211) (actual time=1.599..1.753 rows=20 loops=1)
  ->  Index Scan using books_review_pkey on books_review  (cost=0.44..2860175.68 rows=1886525 width=211) (actual 
time=1.597..1.748 rows=20 loops=1)
        Filter: (library_id = 6)
        Rows Removed by Filter: 3276
Planning Time: 0.119 ms
Execution Time: 1.828 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

ordering: -id
  Total: 0.00
  SQL: 0.00 

      (duration: 0.001)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY "books_review"."id" 
DESC LIMIT 20

Limit  (cost=0.44..30.76 rows=20 width=211) (actual time=0.081..0.161 rows=20 loops=1)
  ->  Index Scan Backward using books_review_pkey on books_review  (cost=0.44..2860175.68 rows=1886525 width=211) 
(actual time=0.079..0.156 rows=20 loops=1)
        Filter: (library_id = 6)
Planning Time: 0.118 ms
Execution Time: 0.219 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [16]:
explain_qs(alexandria_id, "-id")


ordering: -id
  Total: 0.00
  SQL: 0.00 

      (duration: 0.001)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY "books_review"."id" 
DESC LIMIT 20

Limit  (cost=0.44..30.76 rows=20 width=211) (actual time=0.047..0.066 rows=20 loops=1)
  ->  Index Scan Backward using books_review_pkey on books_review  (cost=0.44..2860175.68 rows=1886525 width=211) 
(actual time=0.045..0.063 rows=20 loops=1)
        Filter: (library_id = 6)
Planning Time: 0.102 ms
Execution Time: 0.103 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Once again, there are multiple things to notice:

- The plans for `id` and `-id` are exactly the same for `Public`. They both use the index on `library_id`, then a heap sort

- Both `id` and `-id` orderings use a differnt plan when querying `Alexandria` reviews. It now uses the index on the primary key `books_reviews.id`, and filter rows with the right `library_id`.
  On `-id`, the index is traveled backwards, and only the first 20 rows are kept (because of the `LIMIT 20`)

- This second plan is really fast for `id`, but really slow for `-id`

Conclusion: The plan, and the performance both depend on the data. Always try with a database as close as possible to your live, production database


<br>
<br>
<br>
We can try to add a composite index, as we did for rating:


In [17]:
with use_indexes("review_library_id_pk_idx"):
    explain_qs(alexandria_id, "-id")

ordering: -id
  Total: 0.00
  SQL: 0.00 

      (duration: 0.002)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY "books_review"."id" 
DESC LIMIT 20

Limit  (cost=0.44..30.76 rows=20 width=211) (actual time=0.038..0.056 rows=20 loops=1)
  ->  Index Scan Backward using books_review_pkey on books_review  (cost=0.44..2860175.68 rows=1886525 width=211) 
(actual time=0.037..0.053 rows=20 loops=1)
        Filter: (library_id = 6)
Planning Time: 0.850 ms
Execution Time: 0.095 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

The planner keeps using `books_review_pkey`, even though it does not seem like the right choice. What would happen without this particular index?


In [18]:
from utils.sql import disable_indexes

with use_indexes("review_library_id_pk_idx"), disable_indexes("books_review_pkey"):
    explain_qs(alexandria_id, "-id")


ordering: -id
  Total: 0.01
  SQL: 0.00 

      (duration: 0.004)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY "books_review"."id" 
DESC LIMIT 20

Limit  (cost=0.56..37.54 rows=20 width=211) (actual time=1.665..1.704 rows=20 loops=1)
  ->  Index Scan Backward using review_library_id_pk_idx on books_review  (cost=0.56..3488168.54 rows=1886525 
width=211) (actual time=1.662..1.697 rows=20 loops=1)
        Index Cond: (library_id = 6)
Planning Time: 0.862 ms
Execution Time: 1.760 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

**WARNING** Disabling `books_review_pkey`, which is the primary key index, is probably a very bad idea
<br>
<br>
<br>
<br>
<br>


Another solution, for specific queries, is to use a conditional index, e.g.
`CREATE UNIQUE INDEX review_pk_alexandria_idx ON public.books_review USING btree (id) where library_id = 6;`

This can be a good idea if the the condition is often used. However, the [PostgreSQL doc explicitely says](https://www.postgresql.org/docs/current/indexes-partial.html#INDEXES-PARTIAL-EX4):

> You might be tempted to create a large set of non-overlapping partial indexes (e.g, in our case, with every `library_id`)
> This is a bad idea! Almost certainly, you'll be better off with a single non-partial index, (declared like our `review_library_id_pk_idx`)
> While a search in this larger index might have to descend through a couple more tree levels than a search in a smaller index, that's almost certainly going to be cheaper than the planner effort needed to select the appropriate one of the partial indexes. The core of the problem is that the system does not understand the relationship among the partial indexes, and will laboriously test each one to see if it's applicable to the current query.

> If your table is large enough that a single index really is a bad idea, you should look into using partitioning instead (see Section 5.11). With that mechanism, the system does understand that the tables and indexes are non-overlapping, so far better performance is possible.

We'll still use `review_pk_alexandria_idx` for the rest of this notebook


In [19]:
with use_indexes("review_pk_alexandria_idx"):
    explain_qs(alexandria_id, "-id")

ordering: -id
  Total: 0.00
  SQL: 0.00 

      (duration: 0.003)   EXPLAIN (ANALYZE true) SELECT "books_review"."id", "books_review"."book_id", 
"books_review"."reader_id", "books_review"."rating", "books_review"."comments", "books_review"."written_at", 
"books_review"."library_id" FROM "books_review" WHERE "books_review"."library_id" = 6 ORDER BY "books_review"."id" 
DESC LIMIT 20

Limit  (cost=0.43..1.82 rows=20 width=211) (actual time=1.094..1.119 rows=20 loops=1)
  ->  Index Scan Backward using review_pk_alexandria_idx on books_review  (cost=0.43..131635.41 rows=1886525 
width=211) (actual time=1.092..1.113 rows=20 loops=1)
Planning Time: 1.371 ms
Execution Time: 1.174 ms

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Let's check all the ordering filters now work properly


In [20]:
from books.tests.benchmarks import benchmark_list_reviews

with use_indexes(
    "review_written_at_idx",
    "review_library_id_rating_idx",
    "review_library_id_pk_idx",
    "review_pk_alexandria_idx",
):
    benchmark_list_reviews(
        ["ordered-list-reviews"], library_ids=[public_id, alexandria_id]
    )


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Query params                ┃ Library    ┃ duration ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ ordering=id                 │ Public     │  0.011 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=id                 │ Alexandria │  0.007 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=written_at         │ Public     │  0.030 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=written_at         │ Alexandria │  0.008 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=rating             │ Public     │  0.007 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=rating             │ Alexandria │  0.007 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-id                │ Public     │  0.007 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-id                │ Alexandria │  0.007 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-written_at        │ Public     │  0.021 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-written_at        │ Alexandria │  0.006 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-rating            │ Public     │  0.009 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-rating            │ Alexandria │  0.011 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-written_at,rating │ Public     │  0.018 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-written_at,rating │ Alexandria │  0.010 s │
└─────────────────────────────┴────────────┴──────────┘

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>


---

Ok, now what about filters?


In [21]:
from books.tests.benchmarks import benchmark_list_reviews

with use_indexes(
    "review_written_at_idx",
    "review_library_id_rating_idx",
    "review_library_id_pk_idx",
    # "review_pk_alexandria_idx",
):
    benchmark_list_reviews(["filtered-list-reviews"], library_ids=[alexandria_id])


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Query params               ┃ Library    ┃ duration ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│                            │ Alexandria │  0.009 s │
├────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01 │ Alexandria │  0.015 s │
├────────────────────────────┼────────────┼──────────┤
│ rating__gte=6              │ Alexandria │  0.015 s │
├────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01 │ Alexandria │  0.015 s │
│ rating__gte=6              │            │          │
└────────────────────────────┴────────────┴──────────┘

In [22]:
from books.tests.benchmarks import benchmark_list_reviews

with use_indexes(
    "review_written_at_idx",
    "review_library_id_rating_idx",
    "review_library_id_pk_idx",
):
    benchmark_list_reviews(["complete-list-reviews"], library_ids=[alexandria_id])

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Query params                ┃ Library    ┃ duration ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ ordering=id                 │ Alexandria │  0.013 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=written_at         │ Alexandria │  0.011 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=rating             │ Alexandria │  0.009 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-id                │ Alexandria │  0.007 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-written_at        │ Alexandria │  0.009 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-rating            │ Alexandria │  0.007 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-written_at,rating │ Alexandria │  0.009 s │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.010 s │
│ ordering=id                 │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.011 s │
│ ordering=written_at         │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.010 s │
│ ordering=rating             │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.009 s │
│ ordering=-id                │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.010 s │
│ ordering=-written_at        │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.012 s │
│ ordering=-rating            │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.009 s │
│ ordering=-written_at,rating │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.009 s │
│ ordering=id                 │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.011 s │
│ ordering=written_at         │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.009 s │
│ ordering=rating             │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.011 s │
│ ordering=-id                │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.013 s │
│ ordering=-written_at        │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.011 s │
│ ordering=-rating            │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.011 s │
│ ordering=-written_at,rating │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.013 s │
│ rating__gte=6               │            │          │
│ ordering=id                 │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.023 s │
│ rating__gte=6               │            │          │
│ ordering=written_at         │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.013 s │
│ rating__gte=6               │            │          │
│ ordering=rating             │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-0

Let's try a last index

```sql
CREATE INDEX review_library_id_pk_rating_written_at_idx
    ON books_review USING btree (library_id, id, rating, written_at);
```


In [23]:
from books.tests.benchmarks import benchmark_list_reviews

with use_indexes(
    "review_written_at_idx",
    "review_library_id_rating_idx",
    "review_library_id_pk_idx",
    "review_library_id_pk_rating_written_at_idx",
):
    benchmark_list_reviews(["complete-list-reviews"], library_ids=[alexandria_id])

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Query params                ┃ Library    ┃ duration ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ ordering=id                 │ Alexandria │  0.013 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=written_at         │ Alexandria │  0.012 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=rating             │ Alexandria │  0.010 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-id                │ Alexandria │  0.007 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-written_at        │ Alexandria │  0.010 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-rating            │ Alexandria │  0.010 s │
├─────────────────────────────┼────────────┼──────────┤
│ ordering=-written_at,rating │ Alexandria │  0.010 s │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.010 s │
│ ordering=id                 │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.008 s │
│ ordering=written_at         │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.009 s │
│ ordering=rating             │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.012 s │
│ ordering=-id                │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.011 s │
│ ordering=-written_at        │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.013 s │
│ ordering=-rating            │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.012 s │
│ ordering=-written_at,rating │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.015 s │
│ ordering=id                 │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.012 s │
│ ordering=written_at         │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.010 s │
│ ordering=rating             │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.009 s │
│ ordering=-id                │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.010 s │
│ ordering=-written_at        │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.009 s │
│ ordering=-rating            │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ rating__gte=6               │ Alexandria │  0.012 s │
│ ordering=-written_at,rating │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.011 s │
│ rating__gte=6               │            │          │
│ ordering=id                 │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.014 s │
│ rating__gte=6               │            │          │
│ ordering=written_at         │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-01-01  │ Alexandria │  0.011 s │
│ rating__gte=6               │            │          │
│ ordering=rating             │            │          │
├─────────────────────────────┼────────────┼──────────┤
│ written_at__gte=2022-0

---

Back to Readers per book

Let's try to add an index to try and improve perf for Alexandria, on the previous endpoint


In [24]:
from django.contrib.postgres.aggregates import ArrayAgg


def list_readers_per_book(library_id):
    return dict(
        Book.objects.filter(library=library_id)
        .annotate(reader_names=ArrayAgg("readers__name"))
        .values_list("title", "reader_names")
    )

In [26]:
from utils.sql import use_indexes
from collections import defaultdict 

readers_per_book = defaultdict(list)

with use_indexes("review_book_reader_idx"), perf_counter(message="Tuples", time_sql=True, print_sql=True):
    reviews = Review.objects.filter(library=public_id).values_list(
        "book__title", "reader__name"
    )
    for book_title, reader_name in reviews:
        readers_per_book[book_title].append(reader_name)


with use_indexes("review_book_reader_idx"), perf_counter(time_sql=True, print_sql=True):
    readers_per_book = list_readers_per_book(alexandria_id)


Tuples
  Total: 2.84
  SQL: 2.49 

      (duration: 2.494)   SELECT "books_book"."title", "books_person"."name" FROM "books_review" INNER JOIN 
"books_book" ON ("books_review"."book_id" = "books_book"."id") INNER JOIN "books_person" ON 
("books_review"."reader_id" = "books_person"."id") WHERE "books_review"."library_id" = 10015

Total time
  Total: 9.04
  SQL: 6.71 

      (duration: 6.710)   SELECT "books_book"."title", ARRAY_AGG("books_person"."name" ) AS "reader_names" FROM 
"books_book" LEFT OUTER JOIN "books_review" ON ("books_book"."id" = "books_review"."book_id") LEFT OUTER JOIN 
"books_person" ON ("books_review"."reader_id" = "books_person"."id") WHERE "books_book"."library_id" = 6 GROUP BY 
"books_book"."id"

<br>
<br>
<br>

---

### Index sizes

- The more filters we allow, the more performance issues can arise
- It's not always easy to predict which index will be used
- Indexes are not _always_ the solution, and can take a huge ammount of space
- You need to test on a database as close as possible to your live, production database


In [ ]:
query = """
SELECT
    pg_tables.tablename,
    pg_size_pretty(pg_relation_size('public'::text || '.' || quote_ident(pg_tables.tablename)::text)) AS table_size,
    pg_class.reltuples AS num_rows,
    indexname,
    pg_size_pretty(pg_relation_size('public'::text || '.' || quote_ident(indexrelname)::text)) AS index_size
FROM
    pg_tables
    LEFT OUTER JOIN pg_class ON pg_tables.tablename = pg_class.relname
    LEFT OUTER JOIN (
        SELECT
            pg_class.relname AS ctablename,
            ipg.relname AS indexname,
            indexrelname
        FROM
            pg_index
            JOIN pg_class  ON pg_class.oid = pg_index.indrelid
            JOIN pg_class ipg ON ipg.oid = pg_index.indexrelid
            JOIN pg_indexes ON ipg.relname = pg_indexes.indexname
            JOIN pg_stat_all_indexes psai ON pg_index.indexrelid = psai.indexrelid) AS foo ON pg_tables.tablename = foo.ctablename
WHERE
    pg_tables.schemaname = 'public'
    AND tablename ilike 'books_%'
ORDER BY
    pg_relation_size('public'::text || '.' || quote_ident(pg_tables.tablename)::text) desc,
    pg_relation_size('public'::text || '.' || quote_ident(indexrelname)::text) desc;

"""
with connection.cursor() as cursor:
    cursor.execute(query)
    rows = cursor.fetchall()


table = Table()
table.add_column("Table", style="gold1")
table.add_column("Table size", style="bold green")
table.add_column("Rows count", style="cyan")
table.add_column("Index", style="dark_orange3")
table.add_column("Index size", style="bold green")
for row in map(list, rows):
    row[2] = f"{int(row[2]):,}"
    table.add_row(*map(str, row[:5]))

console.print(table)


┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Table         ┃ Table size ┃ Rows count ┃ Index                                      ┃ Index size ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ books_review  │ 9510 MB    │ 40,025,292 │ review_library_id_pk_rating_written_at_idx │ 1893 MB    │
│ books_review  │ 9510 MB    │ 40,025,292 │ review_library_book_reader_idx             │ 1551 MB    │
│ books_review  │ 9510 MB    │ 40,025,292 │ review_library_id_pk_idx                   │ 1204 MB    │
│ books_review  │ 9510 MB    │ 40,025,292 │ review_book_reader_idx                     │ 1204 MB    │
│ books_review  │ 9510 MB    │ 40,025,292 │ books_review_pkey                          │ 857 MB     │
│ books_review  │ 9510 MB    │ 40,025,292 │ review_written_at_idx                      │ 853 MB     │
│ books_review  │ 9510 MB    │ 40,025,292 │ books_review_book_id_a67a4c60              │ 356 MB     │
│ books_review  │ 9510 MB    │ 40,025,292 │ books_review_reader_id_0438358c            │ 314 MB     │
│ books_review  │ 9510 MB    │ 40,025,292 │ books_review_library_id_e9b4afba           │ 282 MB     │
│ books_review  │ 9510 MB    │ 40,025,292 │ review_library_id_rating_idx               │ 272 MB     │
│ books_review  │ 9510 MB    │ 40,025,292 │ review_rating_idx                          │ 257 MB     │
│ books_review  │ 9510 MB    │ 40,025,292 │ review_pk_alexandria_idx                   │ 43 MB      │
│ books_booktag │ 693 MB     │ 11,407,816 │ unique_book_tag_per_book                   │ 782 MB     │
│ books_booktag │ 693 MB     │ 11,407,816 │ books_booktag_pkey                         │ 257 MB     │
│ books_booktag │ 693 MB     │ 11,407,816 │ books_booktag_book_id_361012b4             │ 173 MB     │
│ books_booktag │ 693 MB     │ 11,407,816 │ books_booktag_library_id_19a4182e          │ 84 MB      │
│ books_book    │ 476 MB     │ 4,001,000  │ unique_book_per_library                    │ 537 MB     │
│ books_book    │ 476 MB     │ 4,001,000  │ book_library_release_date_idx              │ 105 MB     │
│ books_book    │ 476 MB     │ 4,001,000  │ books_book_pkey                            │ 86 MB      │
│ books_book    │ 476 MB     │ 4,001,000  │ books_book_author_id_8b91747b              │ 36 MB      │
│ books_book    │ 476 MB     │ 4,001,000  │ books_book_library_id_9380ba4e             │ 31 MB      │
│ books_book    │ 476 MB     │ 4,001,000  │ book_release_date_idx                      │ 27 MB      │
│ books_person  │ 16 MB      │ 20,100     │ books_person_email_07a23fdf_like           │ 1368 kB    │
│ books_person  │ 16 MB      │ 20,100     │ books_person_email_key                     │ 1352 kB    │
│ books_person  │ 16 MB      │ 20,100     │ books_person_pkey                          │ 456 kB     │
│ books_library │ 568 kB     │ 10,013     │ books_library_pkey                         │ 240 kB     │
└───────────────┴────────────┴────────────┴────────────────────────────────────────────┴────────────┘